In [124]:
import pandas as pd

#a = pd.read_csv('train.tsv', sep='\t', names=['full','split'])['full']
df = pd.read_csv('train.tsv', sep='\t', names=['full','split'])['full'].sample(10000, random_state=42)

In [44]:
#for ind in df.index: 
#    print(df[ind]) 
import nltk
import warnings
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

warnings.filterwarnings('ignore')
nltk.download('punkt') # for tokenizing the text to sentences
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\atefe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\atefe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\atefe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [125]:
from nltk.tokenize.punkt import PunktSentenceTokenizer
tokenized_sentences = []
for ind in df.index: 
    tokenized_sentences.append(df[ind])
print(tokenized_sentences[0])

After the very heavy '' Dragontown '' album Alice decided to return to his roots , his place of birth , Detroit , where he accidently joined in with a festival together with MC5 & Iggy and the Stooges .


In [126]:
from nltk.tokenize import word_tokenize
print(len(tokenized_sentences))
### remove noise ###
stop_words = set(stopwords.words("english"))
ps = PorterStemmer()
lem = WordNetLemmatizer()
filtered_sentences = []
i = 0
for sent in tokenized_sentences:
  curr_sent = []
  t_word = word_tokenize(sent)
  for w in t_word:
    if w not in stop_words and w.isalpha(): # remove the common words
        # sentences.append(ps.stem(w)) # reduce words to their roots (stemming) the result may not be an english word
        curr_sent.append(lem.lemmatize(w,pos="v")) # reduce words to their roots (lemmatization) the result is an english word
  if len(curr_sent) > 0:
    filtered_sentences.append(curr_sent)      

filtered_sentences = list(map(' '.join, filtered_sentences))       
print(len(filtered_sentences))

10000
10000


In [127]:
print(filtered_sentences[0])
warnings.filterwarnings('ignore')
nltk.download('vader_lexicon') # for sentiment analyzer
  
sia = SentimentIntensityAnalyzer()

After heavy Dragontown album Alice decide return root place birth Detroit accidently join festival together Iggy Stooges


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\atefe\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [106]:
sentiment = sia.polarity_scores(filtered_sentences[0])
for k in sorted(sentiment):
    print('{0}: {1}, '.format(k, sentiment[k]), end='')

compound: 0.6597, neg: 0.0, neu: 0.735, pos: 0.265, 

In [128]:
sentiment = [sia.polarity_scores(s) for s in filtered_sentences]
print(len(sentiment))

10000


In [129]:
from statistics import mean

print("Average of the neutral scores:", mean([sent['neu'] for sent in sentiment]))
print("Average of the positive scores:", mean([sent['pos'] for sent in sentiment]))
print("Average of the negative scores:", mean([sent['neg'] for sent in sentiment]))
print("Average of the compound scores:", mean([sent['compound'] for sent in sentiment]))

Average of the neutral scores: 0.8299685
Average of the positive scores: 0.1042937
Average of the negative scores: 0.0657365
Average of the compound scores: 0.09205316


In [130]:
articles = []
sum = 0
for article in filtered_sentences:
    sentiment = sia.polarity_scores(article)
    if sentiment['compound'] < 0.0:
        sum += 1
    if sentiment['pos'] > sentiment['neg']:
        articles.append((article, 'pos'))
    else:
        articles.append((article, 'neg'))    
print(sum)

2304


In [131]:
sum = 0
for (article, cat) in articles:
  if cat == 'pos':
    sum += 1
print(sum)

4306


In [132]:
from nltk.tokenize import word_tokenize 

all_words_set = set(word.lower() for passage in articles for word in word_tokenize(passage[0]))
all_words_list = list(word.lower() for passage in articles for word in word_tokenize(passage[0]))

In [133]:
print(len(all_words_set), len(all_words_list))
filtered_words_list = []
for w in all_words_list:
    if w not in stop_words: # remove the common words
        filtered_words_list.append(lem.lemmatize(w,pos="v"))
print(len(all_words_set), len(filtered_words_list))

29384 167721
29384 160744


In [135]:
word_features = [word for (word,_) in (nltk.FreqDist(filtered_words_list)).most_common(10000)]
word_list = (nltk.FreqDist(filtered_words_list)).most_common(10000)
fdist = nltk.FreqDist(filtered_words_list)
print(word_list[-30:])
# import matplotlib.pyplot as plt
# fdist.plot(10,cumulative=False)
# plt.show()

[('transgender', 2), ('ashbury', 2), ('jimmie', 2), ('counteract', 2), ('illawarra', 2), ('thrill', 2), ('odi', 2), ('hrs', 2), ('antenna', 2), ('rajab', 2), ('prestige', 2), ('academically', 2), ('culpeper', 2), ('hq', 2), ('axé', 2), ('eurodance', 2), ('boca', 2), ('atom', 2), ('valence', 2), ('bathysphere', 2), ('unknowingly', 2), ('outcomes', 2), ('perpendicular', 2), ('okinawa', 2), ('reactivate', 2), ('debug', 2), ('parameters', 2), ('muster', 2), ('phantom', 2), ('giles', 2)]


In [137]:
from random import shuffle
shuffle(articles)
training_articles = articles[:8000]
test_articles = articles[8000:]

In [138]:
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

sa = SentimentAnalyzer()
unigram_features = sa.unigram_word_feats(word_features)
sa.add_feat_extractor(extract_unigram_feats, unigrams=unigram_features)

training_set = sa.apply_features(training_articles)
test_set = sa.apply_features(test_articles)

In [139]:
from nltk.classify import NaiveBayesClassifier as NBC
classifier = sa.train(NBC.train, training_set)

Training classifier


In [140]:
for key, value in sorted(sa.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Evaluating NaiveBayesClassifier results...
Accuracy: 0.57
F-measure [neg]: 0.6684656900539706
F-measure [pos]: 0.38833570412517776
Precision [neg]: 0.5914051841746248
Precision [pos]: 0.5112359550561798
Recall [neg]: 0.7686170212765957
Recall [pos]: 0.3130733944954128
